In [47]:
import os
os.environ["HF_ENDPOINT"] = "https://huggingface.co"

prompt = "Охота на овец"
kwargs = dict(max_new_tokens=100)

### Large model

In [46]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer_l = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model_l = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
from transformers import pipeline
pipe_l = pipeline(task="text-generation", model=model_l, tokenizer=tokenizer_l)

In [42]:
t = pipe_l(prompt, **kwargs, do_sample=False)  # greedy decoding
print(t[0]['generated_text'])

Охота на овец» (1936) и «Смерть в Венеции» (1937).

В 1939 году, после начала Второй мировой войны, он был призван в армию и направлен в Италию. В 1940 году он был ранен в бою и попал в плен. В 1941 году он был освобождён и вернулся в Италию.

В 1943 году он был назначен на должность заместителя начальника штаба полка, а в 1944 году — на должность командира полка. В 1945 году он был назначен на должность командира дивизии,


In [43]:
t2 = pipe_l(prompt, **kwargs, do_sample=True)  # multinomial sampling
print(t2[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец
Елена Лядова Охота на овец
Крошка Монти
Крошка Монти и Крошка Сью
Крошка Монти и Крошка Сью
Крошка Монти и Крошка Сюзанна
Крошка Монти и Крошка Сьюзанн
Крошка Монти и Крошка Сюзанна
Иллюстрации: Александр Голубев
Рисунки Елены Лядовой

http://www.zaycev.net/p


In [44]:
t3 = pipe_l(prompt, **kwargs, num_beams=4, do_sample=True)  # multinomial beam-search sampling
print(t3[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец»

В «Охоте на овец» (1892) Ф. М. Достоевский, как и в «Преступлении и наказании», рисует образ человека, попавшего в ловушку, из которой не может вырваться. Но если в «Преступлении и наказании» человек, попавший в ловушку, не может вырваться, то в «Охоте на овец» человек, попавший в ловушку, может это сделать.

В «Охот


### Small model


In [48]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer_s = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model_s = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
from transformers import pipeline
pipe_s = pipeline(task="text-generation", model=model_s, tokenizer=tokenizer_s)

In [50]:
t = pipe_s(prompt, **kwargs, do_sample=False)  # greedy decoding
print(t[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец
Александр Сергеевич Пушкин




Александр Сергеевич Пушкин

Охота на овец




















































































In [52]:
t = pipe_s(prompt, **kwargs, do_sample=True)  # multinomial sampling
print(t[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец», где фигурирует еще одно имя - «Айзек Азимов и его сыновья», но эта версия еще не стала достоянием широких СМИ, а уже стала бестселлером, а то в эпоху постмодерна даже такие вещи просто невозможно представить!
И да, с таким сюжетом вряд ли кого-то пустят в кинотеатр в видеоряде в оригинале, но мы-то знаем, почему.

И с теми, кто будет слушать эту «Твин Пик


In [53]:
t = pipe_s(prompt, **kwargs, num_beams=4, do_sample=True)  # multinomial beam-search sampling
print(t[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец.





































































































## Fine-tuning

### Pre-process dataset

In [5]:
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
from transformers import GPT2Tokenizer, GPT2Model
gpt2_t = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_m = GPT2Model.from_pretrained("gpt2")
text = "Replace me"
encoded_input = gpt2_t(text, return_tensors="pt")
output = gpt2_m(**encoded_input)
print(output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 0.1629, -0.2166, -0.1410,  ..., -0.2619, -0.0819,  0.0092],
         [ 0.4628,  0.0248, -0.0785,  ..., -0.0859,  0.5122, -0.3939],
         [-0.0644,  0.1551, -0.6306,  ...,  0.2488,  0.3691,  0.0833]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.0719,  2.4170,  0.9660,  ..., -0.4787, -0.3316,  1.7925],
          [-2.2897,  2.5424,  0.8317,  ..., -0.5299, -2.4828,  1.3537],
          [-2.2856,  2.7125,  2.4725,  ..., -1.4911, -1.8427,  1.6493]],

         [[ 0.4796, -0.1131, -1.4854,  ...,  1.1607,  1.8412,  1.3682],
          [-0.7273, -1.1362, -1.0850,  ..., -0.6736,  3.2618,  0.2099],
          [-1.4441, -3.0647, -4.1612,  ..., -1.4788,  3.2718, -0.2803]],

         [[-0.2338, -0.8688,  1.6542,  ..., -1.5964, -1.5636,  1.0931],
          [ 0.3698,  0.4929,  1.4155,  ..., -2.0162, -1.0246,  1.9822],
          [ 0.4509,  1.0144,  0.1189,  ..., -3.1880,  0.4529,  1.3746]],

         ...,

     

In [46]:
gpt2_m.__call__

<bound method Module._call_impl of GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): Laye

In [10]:
from transformers import pipeline
gpt2_p = pipeline(task="text-generation", model=gpt2_m, tokenizer=gpt2_t)
gpt2_p(text)

The model 'GPT2Model' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRF

TypeError: The current model class (GPT2Model) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'GPT2LMHeadModel'}

In [12]:
gpt2_p = pipeline('text-generation', model='gpt2')
gpt2_p(text, max_new_tokens=100)

/Users/vlad/git/vladsaveliev/deeplearning/pretgpt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Replace me, you f****** dog, I\'m your f***** partner, all the time." "I won\'t, I won\'t let you get closer or leave." With that, the girl in the pink uniform left once more, leaving, and went off towards the side of the road on her way to meet the person at the door. "Hey!" Her voice was heavy with anger. "Why am I not happy to see you again? Is that why you\'re still here?!" He looked out'}]

In [18]:
for id in gpt2_t("Replace me")['input_ids']:
    print(gpt2_t.decode(id))

Re
place
 me


In [11]:
from pathlib import Path
drive_path = Path("/Users/vlad/MyDrive/AI")
data_dir = drive_path / "datasets" / "murakami"
with open(data_dir / "murakami_train.txt", "r") as f:
    train_text = f.read()
    print(f"Characters in train text: {len(train_text):,}")
with open(data_dir / "murakami_test.txt", "r") as f:
    test_text = f.read()
    print(f"Characters in test text: {len(test_text):,}")

Characters in train text: 10,127,380
Characters in test text: 27,685


In [3]:
train_ids = tokenizer(train_text)
test_ids = tokenizer(test_text)

In [4]:
print(f"Tokens in train text: {len(train_ids['input_ids']):,}")
print(f"Tokens in test text: {len(test_ids['input_ids']):,}")
# Export to bin arrays
import numpy as np
np.save(data_dir / "train_token_ids.npy", np.array(train_ids['input_ids']))
np.save(data_dir / "test_token_ids.npy", np.array(test_ids['input_ids']))

Tokens in train text: 2,503,609
Tokens in test text: 7,521


In [12]:
train_ids = np.memmap(data_dir / "train_token_ids.npy", mode="r")
test_ids = np.memmap(data_dir / "test_token_ids.npy", mode="r")

class MurakamiDataset(torch.utils.data.Dataset):
    def __init__(self, token_ids: np.memmap):
        self.token_ids = token_ids
        self.n_ctx = model.config.n_ctx

    def __getitem__(self, idx):
        x = torch.tensor(self.token_ids[idx:idx + self.n_ctx])
        y = torch.tensor(self.token_ids[idx + 1:idx + 1 + self.n_ctx])
        item = {"input_ids": x, "labels": y}
        return item

    def __len__(self):
        return len(self.token_ids) - self.n_ctx

train_set = MurakamiDataset(train_ids)
test_set = MurakamiDataset(test_ids)

In [14]:
from transformers import Trainer, TrainingArguments
import evaluate
import numpy as np

# metric = evaluate.load("accuracy")
# 
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=drive_path / "pretgpt" / "murakami_rugpt3small",
        evaluation_strategy="epoch",
        overwrite_output_dir=True,
        save_steps=1000,
        save_total_limit=2,
    ),
    train_dataset=train_set,
    eval_dataset=test_set,
    # compute_metrics=compute_metrics,
)
trainer.train()

/Users/vlad/git/vladsaveliev/deeplearning/pretgpt/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20026952
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7510107
  Number of trainable parameters = 125231616


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.ByteTensor instead (while checking arguments for embedding)

In [12]:
trainer.train()

/Users/vlad/git/vladsaveliev/deeplearning/karpathy/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2501130
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 937926
  Number of trainable parameters = 125231616


TypeError: vars() argument must have __dict__ attribute

dict_keys(['input_ids', 'attention_mask'])


In [8]:
tokenizer.decode(encoded['input_ids'][:100])

'Охота на овец\nЧасть первая\n25.11.1970\nПИКНИК СРЕДИ НЕДЕЛИ\nО ее смерти сообщил мне по телефону старый приятель, наткнувшись на случайные строчки в газете. Единственный абзац скупой заметки он членораздельно зачитал прямо в трубку. Заурядная газетная хроника. Молоденький журналист, едва закончив университет, получил задание и опробовал перо.\nТогда-то и там-то такой-то, находясь за'

In [5]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

t = pipe(prompt, **kwargs, do_sample=True)  # multinomial sampling
print(t[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Охота на овец - это не так уж и сложно, как кажется. Как вы, наверно, уже догадались, эта птица - хищная. У нее тоже есть имя - Одиссей, так же она является хищной птицей. Однажды одиссей спас свою дочь от преследователей и с тех пор он служит своим богам. Но ему по-прежнему предстоит выполнить свою долю на поле чести и справедливости, так что пусть одиссей не думает, что он собирается играть в этой мясорубке с


### Load from checkpoint


In [3]:
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

drive_path = Path("/Users/vlad/MyDrive/AI")
checkpoint_dir = drive_path / "pretgpt/murakami_rugpt3small/checkpoint-2000"

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained(checkpoint_dir)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import pipeline
generate = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Часть первая\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\n•\

In [8]:
result = generate("Часть первая", max_length=1000, do_sample=False)
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Часть первая
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•
•